# Collect results

In [ ]:
import load

results = load.Results(su)

In [ ]:
# for n in [5, 10, 20]:
#   sols = results.collect_and_save(n, linkage_map.keys())
_
# for n in [5, 10, 20]:
#   sols = results.collect_and_save(n, ["rf"])

In [ ]:
import os
import pandas as pd
import scoring
from io_utils import sort_genes
from tqdm import tqdm

RUNS_DIR = os.path.join("..", "runs")
EVAL_DIR = os.path.join("..", "eval")
data_dir = "../data/"

In [ ]:
df = pd.read_csv(data_dir + "data_unscaled.csv")
df
display(df.head())

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scoring import Scoring

S = Scoring()

In [ ]:
def _clean_and_eval(dfs, num_genes, runs):
        for df, run in zip(dfs, runs):
            df['run_id'] = run.split('/')[-1]
        conc = pd.concat(dfs, ignore_index=True)
        sols = sort_genes(conc[[str(i) for i in range(num_genes)]])
        metrics = conc[conc.columns.intersection(scoring.abs_linkages + scoring.log_linkages)]#.reset_index()
        sols = sols.join(metrics)
        
        x = len(sols)
        sols = sols.drop_duplicates(subset=[str(i) for i in range(num_genes)])
        #metrics = sols.apply(lambda x: S.score(x, log=True), axis=1)
        # metrics_log = sols.apply(lambda x: self.scoring.score(x, log=True), axis=1)
        #s#ols = sols.join(metrics)
        #sols = sols.join(conc['run_id'])
        sols = sols.sort_values(by="single_log", ascending=False)
        return sols

In [ ]:
def collect_and_save(num_genes, metrics):
        runs = next(os.walk(RUNS_DIR))[1]
        runs = list(
            filter(
                lambda x: f"_{num_genes}" in x and any(metric in x for metric in metrics),
                runs,
            )
        )
        print(runs)

        runs = [os.path.join(RUNS_DIR, run_dir, "best_sols.csv") for run_dir in runs]
        dfs = []
        for run in tqdm(runs):
            try:
                dfs.append(pd.read_csv(run))
            except FileNotFoundError:
                print('Run is empty: ', run)

        sols = _clean_and_eval(dfs, num_genes, runs)
        sols.to_csv(os.path.join(EVAL_DIR, f'{num_genes}_{"_".join(metrics)}.csv'))
        return sols

In [ ]:
%%time

for n in tqdm(range(4,21)):
    print('-------- ', n, ' -------')
    collect_and_save(n, ['single'])
    
#collect_and_save(20, ['single'])

In [ ]:
%%time

from scoring import linkage_map_log
for n in [5]:
    sols = results.collect_and_save(n, linkage_map_log.keys())

In [ ]:
sols

In [ ]:
%%time

from scoring import linkage_map_log
for n in [10]:
    sols = results.collect_and_save(n, linkage_map_log.keys())
sols